## *Using neural networks to predict perovskite bandgaps*

In this tutorial we will learn how to use the [Keras](https://keras.io/) and [Tensorflow](https://www.tensorflow.org/) libraries to create a neural network regression model that estimates perovskite bandgaps.

You can find another example of neural network regression using Keras in the [TensorFlow Tutorials](https://nanohub.org/tools/tftutorials) nanoHUB tool.

This tutorial uses Python, some familiarity with programming would be beneficial but is not required. Run each code cell in order by hitting "Shift + Enter", or clicking the "Run" button in the menu bar above this notebook. Feel free to modify the code to familiarize yourself with how the code works.

**Outline:**

1. Import libraries
2. Getting data
3. Processing and Organizing Data
4. Creating and training the model
5. Evaluate model training
6. Make some predictions
7. Visualize results

**Get started:** Hit "Shift-Enter" on the code cells to run! 

### Step 1. Import libraries

We first import the relevant libraries. These imports are over four cells:

The first cell imports the [Pandas](https://pandas.pydata.org/) and [Numpy](https://numpy.org/) libraries that we will use to import and convert the data to appropriate formats for the neural network. We will also import modules from the [Scikit-Learn](https://scikit-learn.org/stable/) library that will help pre-process the data

In [ ]:
import pandas as pd
import numpy as np
# featurization
import cmcl
from cmcl import Categories
from spyglass.model_imaging import parityplot
#scikit-learn library imports
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, Normalizer
from sklearn.pipeline import make_pipeline as mkpipe
from sklearn.model_selection import StratifiedShuffleSplit
# ignore all FutureWarnings -- handling coming in a future version of yogi
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

The next cell imports the [Keras](https://keras.io/) and [Tensorflow](https://www.tensorflow.org/) libraries, which we use to construct and train the neural network. The third cell sets the random seed to ensure consistent results every time the notebook is run, an important step in reproducibility

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
tf.random.set_seed(0)

The next cell imports the pyplot module from the matplotlib library for plotting a neural network's version of a Learning Curve

In [ ]:
from matplotlib import pyplot as plt

### Step 2. Getting data

We first load the CSV (comma separated value) file using the Pandas `read_csv()` function. The `set_index` attribute allows us to index the dataframe using our desired set of columns (passed as arguments)

In [ ]:
my = pd.read_csv("./mannodi_data.csv").set_index(["index", "Formula", "sim_cell"])
lookup = pd.read_csv("./constituent_properties.csv").set_index("Formula")

The first element in each row of this dataframe is a chemical composition. We need to convert the raw chemical formula into a numerical representation to develop ML models. This process is generally called featurization, and we will use the `cmcl` library to "featurize" the chemical formula. This library offers convenient dataframe accessors such as `ft.collect.` This function maps the index of raw string formulae to a tabular numerical representation.

In [ ]:
mc = my.ft.comp() # compute numerical compostion vectors from strings
mc = mc.collect.abx() # convenient site groupings for perovskites data

We now group the data points (perovskites) into various categories (pure vs mixed) using the `Categories` class from `cmcl`. These categories are then assigned to the dataframes we loaded earlier, with the label mix. This categorization can help us quantify model performance on a desired subset of data

In [ ]:
#The groupby function groups a dataframe along a desired axis
#This enables operations on the group, such as counting values
mixlog = mc.groupby(level=0, axis=1).count()
mix = mixlog.pipe(Categories.logif, condition=lambda x: x>1, default="pure", catstring="and")
mc = mc.assign(mix=mix).set_index("mix", append=True)
my = my.assign(mix=mix).set_index("mix", append=True)

The categories assigned in the mix variable are now assigned numerical labels using the `OrdinalEncoder()` transformer from Scikit Learn. As an example, the category "pure" is assigned the number 4.

In [ ]:
mixcat = pd.Series(OrdinalEncoder().fit_transform(mix.values.reshape(-1, 1)).reshape(-1),
                     index=mc.index).astype(int)
mixcat.groupby('mix').apply(lambda x: x.to_frame().iloc[0])

### Step 3. Preprocessing and Organizing Data

Having obtained a dataset, we could use all the data to train our model. However, this model would have no guarantee of performing well on an unseen dataset. To mitigate this, we could take one extra step and partition out a "testing" dataset from our data. This dataset will not be used in training the model, and will only be used to judge model performance on unseen data. 

While this is a simple and effective approach, many ML models such as neural networks come with "hyperparameters" that need to be tuned for the data at hand (example: number of layers in a neural network). With one training and testing dataset, we do not have a way to tune the "hyperparameters" of the model. 

There are two solutions to this problem:
1. Carve out an additional "validation" dataset from the training dataset
2. Cross-validation

In approach 1, we define an additional validation set that will be used as "unseen" data to help tune hyperparameters of our model. However, this reduces the amount of data available for training and can be problematic for small datasets. Cross-validation (approach 2) resolves this by using multiple iterations where each iteration uses one chunk of the training data as a validation set. This allows us to tune hyperparameters without wasting too much data.

You can read more about cross-validation [here](https://scikit-learn.org/stable/modules/cross_validation.html). 

Here, we adopt approach 1, i.e, carving a fixed validation set from the training data. For an example of cross-validation, refer to the Random Forest Regressor approach in notebook 2 of this tutorial series.

We use the Scikit Learn Stratified Shuffle Split function to reserve 20% of the data as a test set. You can read more about the Stratified Shuffle Split [here](https://scikit-learn.org/stable/modules/cross_validation.html#stratified-shuffle-split).

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, train_size=0.8, random_state=0)
train_idx, test_idx = next(sss.split(mc, mixcat)) #stratify split by mix categories
mc_tr, mc_ts = mc.iloc[train_idx], mc.iloc[test_idx] #iloc identifies rows in dataframe by index
my_tr, my_ts = my.iloc[train_idx], my.iloc[test_idx]
mixcat_tr, mixcat_ts = mixcat.iloc[train_idx], mixcat.iloc[test_idx]

We now further divide the non-test set of the data into training and validation sets. The first 80% is reserved for training, and the remaining 20% is reserved as a validation set. As before, the validation helps us control model hyperparameters, and monitor for overfitting. 

In [ ]:
#prepare training and validation sets
X = mc_tr.astype(np.float32) #lower the feature precision for more efficient training. These are round fractions anyway 64 bit encoding is just overkill
Y = my_tr.PBE_bg_eV.astype(np.float32) #grab PBE bandgap column from my_tr dataframe

idx = int(0.8*X.shape[0]) #Get a validation set
Xtrain = X.iloc[:idx, :] #80% training
Ytrain = Y.iloc[:idx]
Xval = X.iloc[idx:, :] #20% validation
Yval = Y.iloc[idx:]

#prepare testing set
Xtest = mc_ts.astype(np.float32)
Ytest = my_ts.PBE_bg_eV.astype(np.float32)

In [ ]:
#check for consistency
print (Xtrain.shape, Ytrain.shape, Xval.shape, Yval.shape, Xtest.shape, Ytest.shape)

### Step 4. Creating the Neural Network

We define our neural network architecture in terms of two primary quantities, the number of layers in the network, and the number of "neurons" or nodes in each layer. In this case, we will use two hidden layers, each with 100 nodes. To specify this architecture, we will use the Keras Sequential class.


The layers use [Tanh](https://en.wikipedia.org/wiki/Hyperbolic_functions) (Hyperbolic Tangent) and [ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks) (Rectified Linear Unit) [activation functions](https://en.wikipedia.org/wiki/Activation_function). Click [here](https://towardsdatascience.com/secret-sauce-behind-the-beauty-of-deep-learning-beginners-guide-to-activation-functions-a8e23a57d046) to see and visualize the equations describing these activation functions.

In [ ]:
model = keras.Sequential() #initialize a Sequential model
model.add(keras.Input(shape=(14,))) #Add an input layer, the shape parameter tells how many inputs each data point will have
model.add(layers.Dense(100, activation='tanh')) #Dense defines a fully connected layer, the argument specifies the number of neurons
model.add(layers.Dense(100, activation='tanh')) #activation defines the activation function applied after each layer
model.add(layers.Dense(1, activation='relu')) #Output layer can use a 'relu' activation since outputs are always positive

In [ ]:
model.summary()
#Summary helps view the layers in the model
#and the number of parameters in each layer

Before the model is ready for training, we need to specify a few more settings. These are added during the model's compile step:

- *Loss function:* This measures how accurate the model is during training. We want to minimize this function to "steer" the model in the right direction. Here we use the mean squared error loss function. Click [here](https://towardsdatascience.com/common-loss-functions-in-machine-learning-46af0ffc4d23) to learn more about loss functions.
- *Optimizer:* This decides the optimization technique used to achieve a minimum for the loss function. Here we use the [Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam), click [here](https://climin.readthedocs.io/en/latest/adam.html) to learn more.
- *Epochs:* This decides how long to train the model. One epoch is defined as one iteration over the entire training set, where each iteration loops over all sample batches from the training set. Click [here](https://towardsdatascience.com/epoch-vs-iterations-vs-batch-size-4dfb9c7ce9c9) to learn more about iterations, epochs and batch sizes.

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=1e-3) # Initialize an Adam optimizer with a learning rate of 0.001
model.compile(optimizer=optimizer, loss=keras.losses.MeanSquaredError()) #Compile the model with the Adam optimizer and MSE loss
EPOCHS = 100 #Epoch

### Step 5. Creating the Pipeline

tensorflow estimator objects are mostly scikit learn compliant. So, we will use `model` in a regression pipeline as before.

The pipeline will take care of filling any NaNs with zeros. It will also normalize each record into it's physically significant proportional form as seen in the Exploratory Data Analysis. Click [here](https://en.wikipedia.org/wiki/Feature_scaling) to learn about other types for feature (input) scaling.

In this case, unlike in the random forest example, we will parametrize the Normalizer directly instead of defining a hyperparameter space. The rest of the relevant hyperparameters have already been delt with during the neural network instantiation.

In [ ]:
fillna = SimpleImputer(strategy="constant", fill_value=0.0)
cpipe = mkpipe(fillna, Normalizer('l1'), model)

The pipeline preprocessing steps are used here to prepare the validation set for the sequential estimator. Unfortunately passing the validation set to the pipeline directly is not effective.

In [ ]:
validation_tpl = (cpipe[:-1].fit_transform(Xval),
                  Yval)

We are now ready to train the model. The `fit` method takes in the pandas arrays we arranged earlier. Keras automatically handles backpropogation and updating model weights. To learn more about backpropagation and how neural networks learn, you can click [here](https://www.youtube.com/watch?v=aircAruvnKk) or [here](https://www.youtube.com/watch?v=Ilg3gGewQ5U). It is possible customize this process when engineering advanced networks.

In [ ]:
cpipe.fit(Xtrain, Ytrain, sequential__epochs=EPOCHS, sequential__validation_data=validation_tpl)

At this point, we can check some of the [weights](https://en.wikipedia.org/wiki/Synaptic_weight) from the trained neural network. These weights, in a way, represent the relationship between inputs and outputs.

In [ ]:
weights = cpipe[-1].get_weights()
weights[3]

In [ ]:
#The history object contains the training and validation losses, which we can plot
training_loss = cpipe[-1].history.history['loss']
validation_loss = cpipe[-1].history.history['val_loss']

### Step 6. Evaluate model training

We use pyplot from [Matplotlib](https://matplotlib.org/) to plot the "Learning Curve", which is a plot that shows the evolution of training and validation loss over epochs. We expect the training and validation losses to go down if the training went well. More importantly, if the validation loss goes up after a while, we know that the model is overfitting. In Keras, this overfitting can be prevented using the `EarlyStopping()` functionality, click [here](https://keras.io/api/callbacks/early_stopping/) to learn more.

In [ ]:
plt.plot(training_loss, 'b-', label='training')
plt.plot(validation_loss, c='orange', label='validation')
plt.xlabel("Epoch")
plt.ylabel("Loss value (eV^2)")
plt.legend()
plt.show()

At this stage, if we are satisfied with the model training, we can call the `model.evaluate()` function on the training, validation, and test sets to measure model performance. If we wish to improve training, we would go back and change parameters in the building of the model such as running the training for more epochs or adding more layers to the model. The effect of these changes will be tracked by the learning curves.

In [ ]:
train_tpl = (cpipe[:-1].fit_transform(Xtrain),
             Ytrain)
test_tpl = (cpipe[:-1].fit_transform(Xtest),
            Ytest)

#The model.evaluate() function evaluates the model on the training, validation and testing datasets
mse_train = cpipe[-1].evaluate(*train_tpl)
mse_val = cpipe[-1].evaluate(*validation_tpl)
mse_test = cpipe[-1].evaluate(*test_tpl)

print(f"Averaged Squared Error For each Data Partition:\nTrain:\t\t{mse_train}\nValidation:\t{mse_val}\nTest:\t\t{mse_test}")

We can save the model into an h5 format training by using the `model.save()` function. This saved model can be reloaded using the `load_model()` function.

This tool is running in a read only filesystem, so this code is just for reference.

`model.save('./Models/nn_c_bg.h5')`

`load_model = keras.models.load_model('./Models/nn_c_bg.h5')`

### Step 7. Make some predictions

We can call the `model.predict()` function to make predictions. We will use this function to make predictions on the train, validation and test sets. We expect good predictions for the training and validation sets, but the predictions on the test sets are unknown

In [ ]:
Y_pred_tr = cpipe.predict(Xtrain)
Y_pred_val = cpipe.predict(Xval)
Y_pred_test = cpipe.predict(Xtest)

### Step 8. Visualize results

Finally, Parity Plots show how predictions compare to the true values. We see that the model does a reasonable job at predicting band gaps for each of the train, validation, and test sets.

Because neural networks can potentially learn any function, perhaps there is some chance this is a model of the physical relation between composition and bandgap.

But probably, with such a simply linear architecture, this model only captures statistical correlations limited to this data. Nevertheless, the model's potential demands more study.

In [ ]:
plt.plot(Ytrain, Y_pred_tr, 'ro', label="training")
plt.plot(Yval, Y_pred_val, 'bo', label="validation")
plt.plot(Ytest, Y_pred_test, 'go', label="testing")
plt.plot(Ytrain, Ytrain, 'k-')
#x = np.linspace(min(Ytrain), max(Ytrain), 1000)
#plt.plot(x, x, 'k-')
plt.gca().set_aspect(1.0)
plt.xlabel("Ground truth bandgap (eV)")
plt.ylabel("Predicted bandgap (eV)")
plt.legend()
plt.show()

A second parity plot shows how this model handles each alloy type in the data as well.

In [ ]:
p, data = parityplot(cpipe, mc_tr, my_tr.PBE_bg_eV.to_frame(), aspect=1.0, hue="mix")